In [50]:
import requests
from bs4 import BeautifulSoup


url = 'https://3w.huanqiu.com/a/c36dc8/3xqGPRBcUE6?agt=8'
url = 'https://baijiahao.baidu.com/s?id=1689953863549414325'
# 提取信息，并转UTF-8避免乱码
html = requests.get(url, timeout = 10)
content = html.content
# 创建BeautifulSoup对象
soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
text = soup.get_text()

In [63]:
# 用jieba切割文本信息，便于用词性提取信息
import jieba
import jieba.posseg as pseg
words = pseg.lcut(text)

# 人名词性：人名（nr），地点（ns）
news_person = [word for word, flag in words if flag == 'nr']
news_place = [word for word, flag in words if flag == 'ns']

from collections import Counter

c = Counter(news_person)
# 名字出现的频率
c.most_common(3)

[('马斯克', 74), ('侠', 3), ('高超', 1)]

In [32]:
import re
# 抓取正文内容，对非中文内容进行清洗
text = re.sub('[^[\u4e00-\u9fa5。，！：、]{3,}', '', text)

'\n马斯克才是时间管理大师[[[[[[[];百度首页登录个人中心帐号设置意见反馈退出马斯克才是时间管理大师字母榜发布时间：年1月，加冕新首富、猎鹰火箭一箭星、国产落地，埃隆·马斯克的成就一口气打了好几个勾，短时间内这么多收获，离不开他高超的时间管理术。作为这个世界上最成功最忙碌的男人之一，马斯克管理7家公司，还养育了6个孩子——普通人就算有三头六臂恐怕都忙不过来，马斯克到底是怎么做到的？首先是工作法，为任务设置时长——凡是醒着状态，马斯克会以5分钟为单位来安排日程，一天时光就切成了几百个“时间方糖”。到工作时间，每天的最低标准是处理最关键的事务，包括最紧要工作、需要见面的关键人物和必须参加的关键活动。马斯克接下来的非工作事务，马斯克会压缩在最短时间内完成，包括花5分钟吃饭，一周为约会安排10小时，小便也要3秒钟解决等。一心二用甚至三用，是马斯克管理时间的一大特点，他会在开会时用手机打理生意，在检查发票时回复邮件，有时抱孩子坐在腿上玩耍还会抽空回份邮件年美国连线杂志对马斯克花费时间进行有趣的对比换算，得出的结论是：马斯克一年干了别人八年的工作量。造火箭，对比蓝色起源，耗时比例造自动驾驶汽车，特斯拉对比福特，耗时比造超级高铁，马斯克对比加州政府的项目，耗时比挖隧道，对比正常地铁，耗时比交付平民价位的电动汽车，从立项到交付，耗时是通用汽车的更快的汽车生产线，通常汽车组装需要3分钟，特斯拉机器人生产线10秒搞定，耗时比登陆火星，按规划马斯克想在年就送人类上火星，要到，马斯克又领先；以道御术，马斯克做好技术层面时间管理的同时，又是什么赋予了他对时间的驾驭感？「第一性原理」是马斯克挂在嘴边的理论之一，他要确保自己了解极大多数事情背后的基础科学原理或规则——这可以减少他花费大量时间去学习新概念，认知事物便有了一个平滑上扬的认知曲线。真·时间管理大师往往分成两类，一类是以比尔·盖茨为首的低调劳模，为了省时间，盖茨把果珍当饭吃，著名理论是：“冲果珍的时间可以用来工作，因为你身体里有水。”第二类则是马斯克这种工作情感双栖发展的达人。纪录片《走进比尔：解码比尔盖茨》就讲过比尔·盖茨的一天：早上6点起床，8点参加人工智能技术报告，9点和泰拉能源公司开会，10点半是微软董事会，中午留半小时午餐，和巴菲特打电话，12点半开卫生会议，接下来还要接受记者采访、去实验室……繁忙的工作也没耽误情感生

In [40]:
from wordcloud import WordCloud
# 去掉停用词
def remove_stop_words(w, filter_word=[]):
    for stop_word in filter_word:
        w = w.replace(stop_word, '')
    return w

def create_word_cloud(f, filter_word):
    f = remove_stop_words(f, filter_word)
    seg_list = jieba.lcut(f)
    cut_text = ' '.join(seg_list)
    wc = WordCloud(
    max_words=100,
        width=2000,
        height=1200,
        font_path='./msyh.ttf'
    )
    wordcloud = wc.generate(cut_text)
    wordcloud.to_file("../result/wordcloud.jpg")

In [46]:
# 输出重要的句子
from textrank4zh import TextRank4Keyword, TextRank4Sentence

# 获得关键字
def get_key_word(text,topN,word_min_len=2):
    tr4w = TextRank4Keyword()
    tr4w.analyze(text=text, lower=True, window=3)
    return [item.word for item in tr4w.get_keywords(topN, word_min_len=2)]
# 获得摘要 
def get_sentences(text, num=3):
    tr4s = TextRank4Sentence()
    tr4s.analyze(text=text, lower=True, source = 'all_filters')
    return [item.sentence for item in tr4s.get_key_sentences(num=num)]

In [45]:
# 文章关键字
get_key_word(text, 3, 2)

['马斯克', '时间', '工作']

In [47]:
# 文章摘要
get_sentences(text, 3)

['马斯克则在起床后第一时间投入工作，睡醒就开始争分夺秒地刷邮件，，他会处理那些紧急而重要的工作电邮',
 '首先是工作法，为任务设置时长——凡是醒着状态，马斯克会以5分钟为单位来安排日程，一天时光就切成了几百个“时间方糖”',
 '马斯克接下来的非工作事务，马斯克会压缩在最短时间内完成，包括花5分钟吃饭，一周为约会安排10小时，小便也要3秒钟解决等']

In [48]:
# 生成词云
create_word_cloud(text, filter_word=['时候', '没有', '他', '在', '的', '也', '是'])